<a href="https://colab.research.google.com/github/obrookes/Natural-Language-Processing/blob/master/NLP_SMS_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flair

**Basics**

**Text classification: ham or spam**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%mkdir nlp
%ls "/content/gdrive/My Drive/nlp"

!cp /content/gdrive/My\ Drive/nlp/spam.csv ./nlp
%cd nlp

spam.csv
/content/nlp


In [5]:
import pandas as pd
data = pd.read_csv("spam.csv", encoding='latin-1').sample(frac=1).drop_duplicates()

This will remove some duplicates from our dataset, shuffle it (randomise rows) and split the data into train, dev and test sets using the 80/10/10 split.

In [6]:
data = data[['v1', 'v2']].rename(columns={"v1":"label", "v2":"text"})
 
data['label'] = '__label__' + data['label'].astype(str)

data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [11]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = './'

# load corpus containing training, test and dev data
corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train.csv')

2020-07-22 16:30:12,552 Reading data from .
2020-07-22 16:30:12,555 Train: train.csv
2020-07-22 16:30:12,557 Dev: dev.csv
2020-07-22 16:30:12,559 Test: test.csv


In [12]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path


word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-07-22 16:30:18,793 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpvgndxnxj


100%|██████████| 19689779/19689779 [00:03<00:00, 5375476.53B/s]

2020-07-22 16:30:23,583 copying /tmp/tmpvgndxnxj to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt


2020-07-22 16:30:23,618 removing temp file /tmp/tmpvgndxnxj
2020-07-22 16:30:39,418 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpfutmk02p


...

2020-07-22 16:30:54,847 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  
100%|██████████| 4652/4652 [00:04<00:00, 1137.01it/s]

2020-07-22 16:30:59,219 [b'ham', b'spam']
2020-07-22 16:30:59,227 ----------------------------------------------------------------------------------------------------
2020-07-22 16:30:59,229 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=

2020-07-22 16:31:06,626 epoch 1 - iter 13/130 - loss 0.33866110 - samples/sec: 61.97
2020-07-22 16:31:13,515 epoch 1 - iter 26/130 - loss 0.30361806 - samples/sec: 61.09
2020-07-22 16:31:20,399 epoch 1 - iter 39/130 - loss 0.25304065 - samples/sec: 61.10
2020-07-22 16:31:26,650 epoch 1 - iter 52/130 - loss 0.22133297 - samples/sec: 72.23
2020-07-22 16:31:34,000 epoch 1 - iter 65/130 - loss 0.20811637 - samples/sec: 57.22
2020-07-22 16:31:39,822 epoch 1 - iter 78/130 - loss 0.19246973 - samples/sec: 72.42
2020-07-22 16:31:45,347 epoch 1 - iter 91/130 - loss 0.17475055 - samples/sec: 76.12
2020-07-22 16:31:52,119 epoch 1 - iter 104/130 - loss 0.16299054 - samples/sec: 66.32
2020-07-22 16:31:58,785 epoch 1 - iter 117/130 - loss 0.15394092 - samples/sec: 63.18
2020-07-22 16:32:03,724 epoch 1 - iter 130/130 - loss 0.14524899 - samples/sec: 85.06
2020-07-22 16:32:03,870 ----------------------------------------------------------------------------------------------------
2020-07-22 16:32:03,87

{'dev_loss_history': [0.08764049410820007,
  0.22912871837615967,
  0.10419827699661255,
  0.19496899843215942,
  0.051648594439029694,
  0.054545219987630844,
  0.05132061243057251,
  0.1713988482952118,
  0.05603422224521637,
  0.06960493326187134],
 'dev_score_history': [0.9857,
  0.9755,
  0.9838,
  0.9691,
  0.9904,
  0.9923,
  0.9914,
  0.981,
  0.9914,
  0.9923],
 'test_score': 0.9914,
 'train_loss_history': [0.14524899281812115,
  0.07431411712154491,
  0.06878707946833366,
  0.05538795283780648,
  0.053463201580318406,
  0.04608076030675035,
  0.03993450147637095,
  0.043127408150870065,
  0.03754919711858607,
  0.03594464436397315]}

In [19]:
classifier = TextClassifier.load('./best-model.pt')

# create example sentence
sentence = Sentence('click here for free stuff...')

# predict class and print
classifier.predict(sentence)

print(sentence.labels)


2020-07-22 17:00:39,882 loading file ./best-model.pt


TypeError: ignored